# An Analysis of Traffic Accidents in Madrid Involving Bicycles

Project Description goes here

## Getting the Datasets

In [ ]:
!mkdir ../datasets
%cd ../datasets
!curl -L -o dataset_#1.xlsx https://datos.madrid.es/egob/catalogo/300110-[1-17:2]-accidentes-bicicleta.xlsx
%cd ../notebooks

## Importing the Datasets with Pandas
General imports.

## Importing the Datasets to Pandas

Import each of the datasets and store them in a list. Then, concat the dfs and show head().

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Number of victims involved.

In [ ]:
print(f"Victims in accidents involving bicycles between 2010 and 2018: {df.shape[0]}")

## Translating the DataFrame
Show columns.

In [ ]:
df.columns

Translate columns.

In [ ]:
eng_cols = ["Date", "Hour Range", "Day of Week", "District", "Location", "No.", "Report No.", "Weather: Hail", "Weather: Ice","Weather: Rain", "Weather: Fog", "Weather: Dry", "Weather: Snow", "Road: Wet", "Road: Oil", "Road: Mud", "Road: Gravel", "Road: Ice", "Road: Dry and clean", "Victims", "Accident Type", "Vehicle Type", "Person Type", "Gender", "Harmfulness", "Age Range", "* Victims"]

df.columns = eng_cols

Some colums dont convey much info etc etc. For example

In [ ]:
df.query("`Vehicle Type`.str.strip() != 'BICICLETA'")

Lets drop unnecessary info.

In [ ]:
df.drop(columns=["Day of Week", "Report No.", "Weather: Hail", "Vehicle Type", "* Victims"], inplace=True)
df.columns

Now let's translate the values. See the unique values in the columns we want to translate:

In [ ]:
for i in range(5, df.shape[1]):
    print(f"{df.iloc[:, i].name}: {df.iloc[:, i].unique()}")

Strip the strings

In [ ]:
df[["Accident Type", "Person Type", "Harmfulness", "Age Range"]] = df[["Accident Type", "Person Type", "Harmfulness", "Age Range"]].transform(lambda x: x.str.strip())

Now create translation dictionaries to map to each column.

In [ ]:
# Yes/No dict for the first columns
yn_dict = {"SI": "Yes", "NO": "No"}

# Translate first columns
for i in range(5, 17):
    df.replace({df.iloc[:,i].name: yn_dict}, inplace=True)

# English translation for the rest of the columns
acc_type_eng = ["Crash with static object", "Fall off bicycle", "Run over", "Double collision", "Fall off motorcycle", "Bus passenger fall", "Multiple collision", "Other causes", "Fall off moped", "Overturn"]
person_type_eng = ["Driver", "Accompanying", "Witness"]
gender_eng = ["W", "M", "Not assigned"]
harmfulness_eng = ["Seriously injured", "Slightly injured", "Uninjured", "Not assigned", "Killed"]
age_range_eng = ["21-24", "25-29", "18-20", "30-34", "45-49", "65-69", "35-39", "55-59", "50-54", "40-44", "10-14", "15-17", "6-9", "60-64", "0-5", "75+", "Unknown", "70-74"]
eng_list = [acc_type_eng, person_type_eng, gender_eng, harmfulness_eng, age_range_eng]

# Create the translation dictionaries and translate the rest of the columns
for i in range(18, df.shape[1]):
    esp_list = df.iloc[:, i].unique()
    dict = {k: v for k, v in zip(esp_list, eng_list[i - 18])}
    df.replace({df.iloc[:,i].name: dict}, inplace=True)

## Handling Missing Values
Let's check for null entries.

In [ ]:
df.isnull().any()

There is only null values in the "Victims" column. Let's see how many accident entries have a null value.

In [ ]:
df["Victims"].isnull().sum()

This is a significant amount of data ($\approx 13$%), so fill NaN values.
Let's see wich type of person was involved in this accidents.

In [ ]:
df[df["Victims"].isnull()].groupby("Person Type")["Date"].count()

For the sake of simplicity, let's suppose drivers were the only ones injured in their accidents, and accompanyings were involved in accidents with two victims. We'll discard all witnesses as they probably only reported a crash but weren't involved in it.

In [ ]:

df.loc[df["Person Type"] == "Driver", "Victims"] = df.loc[df["Person Type"] == "Driver", "Victims"].fillna(1)

df.loc[df["Person Type"] == "Accompanying", "Victims"] = df.loc[df["Person Type"] == "Accompanying", "Victims"].fillna(2)

df.drop(index=df.query("`Person Type` == 'Witness'").index, inplace=True)

## Rearanging the DataFrame
Rearange the weather and road conditions data so it is contained in two columns

In [ ]:
df["Weather"] = df.apply(lambda row: row.iloc[5:11][row.iloc[5:11] == "Yes"].index[0].split(" ")[-1], axis="columns")

df["Road condition"] = df.apply(lambda row: row.iloc[11:17][row.iloc[11:17] == "Yes"].index[0].split(" ")[-1], axis="columns")

The second operation gives an error because:

In [ ]:
df.query("`Road: Wet` == 'No' & `Road: Oil` == 'No' & `Road: Mud` == 'No' & `Road: Gravel` == 'No' & `Road: Ice` == 'No' & `Road: Dry and clean` == 'No'")["Date"].count()

Lets suppose those days were normal:

In [ ]:
df[(df["Road: Wet"] == "No") & (df["Road: Oil"] == "No") & (df["Road: Mud"] == "No") & (df["Road: Gravel"] == "No") & (df["Road: Ice"] == "No") & (df["Road: Dry and clean"] == "No")]["Road: Dry and clean"] = "Yes"

In [ ]:
df_list = []
for i in range(9):
    curr_df = pd.read_excel("../datasets/dataset_" + str(2*i + 1) + ".xlsx")
    df_list.append(curr_df)
    
df = pd.concat([df_list[i] for i in range(9)], ignore_index=True)
df.head()